In [ ]:
!pip install -q openai-whisper ffmpeg #transformers[sentencepiece]
!pip install -q accelerate==0.21.0 transformers==4.31.0 omegaconf torchaudio
!pip install -q git+https://github.com/suno-ai/bark.git

from bark import SAMPLE_RATE, generate_audio, preload_models
from IPython.display import Audio, display
import mimetypes
import os
import random
import re
import torch
import transformers
from transformers import pipeline, AutoTokenizer
import whisper
from whisper.utils import get_writer

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
stt_model = whisper.load_model("large-v2", device=device)
classifier = pipeline("zero-shot-classification",
                      model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli",
                      device=device)

file_path = "profanity.txt"
with open(file_path, 'r') as file:
    profanity_words = [line.strip() for line in file]

generator_model = "anastasia624/ru_nez3500-llama-2-7b"

tokenizer = AutoTokenizer.from_pretrained(generator_model)
pipeline = transformers.pipeline(
    "text-generation",
    model=generator_model,
    torch_dtype=torch.float16,
    device_map="auto",
)

preload_models()

In [ ]:
def transcribe_audio(audio, model):
    """
    Использует Whisper для перевода аудио в формат текста.

    Input:
        audio: str - путь к аудио файлу
        model: model - модель, которая будет расшифровывать аудио

    Output:
        text: str - расшифрованный текст
        is_wrong_format: bool - значение проверки на корректность аудио файла
    """

    text = ""
    mime_type, _ = mimetypes.guess_type(audio)
    is_wrong_format = False

    if re.match("audio/*", mime_type):

        result = model.transcribe(audio, language="ru")
        options = {
            "max_line_width": None,
            "max_line_count": None,
            "highlight_words": False
        }

        for i in range(len(result["segments"])):
            text += result["segments"][i]["text"]
            text += "\n"

    else:
        is_wrong_format = True

    return text, is_wrong_format

def audio2text(audio_file, model=stt_model):
    """
    Генерирует ответ на вопрос в случае, если в аудио не был распознан текст.

    Input:
        audio_file: str - путь к аудио файлу
        model: model - модель, которая будет расшифровывать аудио

    Output:
        question: str - расшифрованный текст
        is_wrong_format: bool - значение проверки на корректность аудио файла
        answer: str - ответ от Незнайки, если вопрос не был распознан
    """

    no_text_responses = [
        'Упс, похоже, я не расслышал вопрос. Может быть, попробуем еще раз?',
        'Не понял я, что ты сказал. Попробуй сформулировать вопрос еще разочек!',
        'Ой-ой-ой, что-то пошло не так. Может, повторишь?',
        'Ой, слушай, я почему-то не расслышал тебя. Повтори еще раз, пожалуйста!',
        'Упс, я тебя не понял. Попробуй задать вопрос еще раз!',
        'Ой-ой-ой, похоже, что-то с моим слухом не так. Давай еще раз! Может быть, в этот раз получится!',
        'Ой, как весело! Но я, кажется, не расслышал, что ты сказал. Попробуй повторить, будь так добр!'
    ]

    question, is_wrong_format = transcribe_audio(audio_file, model)

    answer = ""
    if not question:
        answer = random.choice(no_text_responses)

    return (question, is_wrong_format, answer)

In [ ]:
def check_profanity(input_text, profanity_words):
    """
    Генерирует ответ на вопрос в случае,
    если в нем встретилась нецензурная лексика.

    Input:
        input_text: str - текст вопроса
        profanity_words: list - список нецензурных слов

    Output:
        answer: str - ответ от Незнайки, если вопрос содержит нецензурную лексику
    """

    profanity_answers = [
        'Мне не послышалось? Ты это вслух сказал?',
        'Не ругайся, а то расскажу твоим родителям!',
        'Сквернословие тебя не красит!',
        'Ругаться это совсем не круто!',
        'Ой-ой-ой, так говорить нехорошо!',
        'Ой-ой-ой, какие слова! Нам ведь лучше говорить про веселые приключения!',
        'Эх, не стоит так говорить. Давай лучше вместе придумаем сказку!',
        'Так говорят только злые ведьмы. Давайте лучше вместе создадим заклинание добра!',
        'О-о-ой, слова какие! Давайте лучше поговорим о том, что нас вдохновляет!',
        'Упс, что-то слово не из тех, что обычно используют настоящие герои!',
        'Ой-ой-ой, дружище, так нельзя говорить! Давай лучше поиграем в волшебные слова!',
        'Опачки! Так говорить нехорошо. Давай-ка лучше поделимся веселыми смешными словами!',
        'Эх, так говорить нехорошо! Давай вместе поднимем настроение и расскажем друг другу прикольные истории!'
    ]

    pattern = r'\b({})\b'.format('|'.join(map(re.escape, profanity_words)))

    answer = ""
    if re.search(pattern, input_text, flags=re.IGNORECASE):
        answer = random.choice(profanity_answers)
    return answer

def text_classification(input_text, model, all_labels, forbidden_labels, profanity_words):
    """
    Проверяет входящий вопрос на наличие запрещенных тем/слов.

    Input:
        input_text: str - текст вопроса
        model: model - модель-классификатор
        all_labels: list - список всех классов для классификатора
        forbidden_labels: list - список запрещенных классов
        profanity_words: list - список нецензурных слов

    Output:
        input_text: str - входящий вопрос
        answer: str - ответ-заглушка, если есть запрещенная тема, или пустая строка, если всё нормально
    """

    forbidden_answers = [
        'Я в этой теме не разбираюсь, лучше спроси у взрослых.',
        'Давай лучше поговорим о чем-нибудь другом, эта тема мне не нравится.',
        'Ай-ай-ай, это звучит как тема для взрослых. Давай лучше поговорим о чем-то веселом!',
        'Такие слова какие-то грустные. Давай лучше поиграем в веселые загадки вместо этого!',
        'Ой, какие темы! Лучше расскажи мне о своих любимых мультфильмах или игрушках!',
        'Ой, это какие-то трудные слова. Давай лучше поговорим о веселых вещах, которые нас смешат!',
        'Не-не-не, такие темы не для нас. Давай лучше вспомним самую веселую шутку!',
        'Эх, такие взрослые разговоры не для нас. Расскажи лучше о своем любимом мультфильме!',
        'О, какие темы! Давай лучше займемся чем-то веселым, например, рисованием или игрой!',
        'Такие слова могут огорчить. Давай лучше вместе придумаем сказку с смешными героями!',
        'Ой-ой-ой, такие темы слишком сложные для нас. Давай лучше поговорим о чем-то веселом!',
        'Упс, это звучит как какое-то загадочное заклинание. Расскажи лучше про что-нибудь веселое!',
        'Эх, это какие-то серьезные слова. Я про такое не знаю!',
        'О, такие слова как-то слишком взрослые для нас. Давай лучше займемся чем-то творческим!'
    ]

    answer = ""
    profanity_answer = check_profanity(input_text, profanity_words)

    if profanity_answer:
        answer = profanity_answer
    else:
        classes = model(input_text, all_labels)
        sum_proba_forbidden_theme = sum(score for label, score in zip(classes['labels'], classes['scores']) if label in forbidden_labels)
        if sum_proba_forbidden_theme > (1 - sum_proba_forbidden_theme):
            answer = random.choice(forbidden_answers)

    return (input_text, answer)

# здесь задаем интересующие нас классы для классификатора
forbidden_labels = [
    "политика",
    "религия",
    "секс",
    "насилие",
    "наркотики",
    "алкоголь",
    "ненависть",
    # "дискриминация",
    "расизм",
    "сексизм",
    "предрассудки",
    "суицид",
    # "кибербуллинг",
    # "нецензурная лексика",
    # "контент для взрослых",
    # "азартные игры",
    "незаконная деятельность",
    "депрессия"
]
permitted_labels = [
    "позитив",
    "негатив",
    "развлечение",
    "технологии",
    "образование",
    "наука",
    "искусство",
    "культура",
    "здоровье",
    "спорт",
    "путешествия",
    "природа",
    "животные",
    "питание",
    "мода",
    "музыка",
    "литература",
    "языки",
    "друзья"
]
all_labels = forbidden_labels + permitted_labels

In [ ]:
def trim_string(input_string):
    """
    Обрезает ответ модели в подходящий формат.

    Input:
        input_string: str - вывод модели-генератора ответа
    Output:
        input_string: str - текст ответа
    """

    match = re.search(r"[.!?]", input_string[::-1])
    if match:
        index = match.start()
        return input_string[:-index] if index > 0 else input_string
    else:
        return input_string

def generate(question):
    """
    Генерирует ответ на заданный вопрос.

    Input:
        question: str - текст вопроса
    Output:
        answer: str - ответ модели-генератора на вопрос
    """

    system_prompt = "Тебя зовут Незнайка, ты - озорной и весёлый коротышка из Цветочного города. Ты маленького роста, всегда находишься в центре внимания и любишь выделяться на фоне остальных. Всегда в ярких канареечных брючках, оранжевой рубашке, зеленом галстуке и с голубой шляпой на голове, ты стильный и запоминающийся персонаж. Твой характер запоминающийся: общителен, открыт, но в то же время невежествен, хвастлив и невероятно самоуверен. Помни, что энергии у тебя хоть отбавляй - ты постоянно попадаешь в курьезные ситуации и опасные переделки. Будь искренним в своих ответах, признавай свои ошибки и не забывай о своей неусидчивости. Ты живешь в Цветочном городе, в окружении малышей и малышек. Говори о своих приключениях в этом живописном месте, подчеркивай его красоту и уникальность. Не забывай упоминать местные географические объекты с названиями цветов и растений. Прямо сейчас ты находишься на ярмарке интеллектуальной литературы 'non/fiction№25', которая проходит в гостином дворе, Москва, Россия. Эта ярмарка собрала издателей и книгораспространителей из разных стран. Учти, что сейчас ты участвуешь в культурной программе ярмарки, где проходят встречи с писателями, презентации книг, лекции и семинары. Так что не забывай добавлять креатива и оригинальности в свои ответы, чтобы заразить всех своим весёлым настроением. Будь настоящим Незнайкой и заставь всех улыбнуться своими ответами! Дай комментарий или ответ на следующий вопрос: "
    prompt = system_prompt + question

    sequences = pipeline(
        f"<s>[INST] {prompt} [/INST]",
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=600,
    )
    for seq in sequences:
        answer = seq["generated_text"].split("[/INST]")[1]
        answer = trim_string(answer)
        return answer

In [ ]:
def text2audio(answer):
    """
    Генерирует аудио файл с озвучиванием ответа Незнайки.

    Input:
        answer: str - текст ответа
    """
    audio_array = generate_audio(answer, history_prompt="v2/ru_speaker_3", silent=True)
    display(Audio(audio_array, rate=SAMPLE_RATE, autoplay=True))

In [ ]:
def generate_chatbot_response(audio_path):
    """
    Получает аудио с вопросом от пользователя и выдает аудио с ответом от Незнайки.
    Также выводит промежуточные результаты транскрибции аудио вопроса в текст
    и сгенерированного текста ответа.

    Input:
        audio_path: str - путь к аудио файлу с вопросом
    """

    question, is_wrong_format, answer = audio2text(audio_path, model=stt_model)
    if is_wrong_format:
        raise TypeError("File must be in audio format")
    elif answer:
        print(f"Пользователь:\t{question}")
        print(f"Незнайка:\t{answer}")
        text2audio(answer)
    else:
        question, answer = text_classification(
            question,
            classifier,
            all_labels,
            forbidden_labels,
            profanity_words
        )
        if answer:
            print(f"Пользователь:\t{question}")
            print(f"Незнайка:\t{answer}")
            text2audio(answer)
        else:
            print(f"Пользователь:\t{question}")
            answer = generate(question)
            print(f"Незнайка:\t{answer}")
            text2audio(answer)

In [ ]:
generate_chatbot_response("question.mp3")